In [2]:
import pandas as pd
import re
import pubchempy as pcp
import pandas as pd
import rdkit.Chem
import rdkit.Chem.AllChem
import rdkit.DataStructs

#### get whole DrugComb dataset

In [3]:
df_dc = pd.read_csv("/home/cx79/scratch60/NCI/DB/summary_v_1_5.csv",encoding = "ISO-8859-1",dtype = str)
## remove na
df_dc_nadrop = df_dc.dropna(how='any')
subdf_dc = df_dc_nadrop

In [4]:
subdf_dc.groupby(by=["combined"])['Synergy'].transform(any)

,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,ic50_row,ic50_col,...,S_mean,S_max,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name
19037,452434,lonidamine,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,0.0760400699439285,4.49697005007709,...,39.06100000000001,24.909500000000005,0.409634887137493,-11.702283257798,-1.25228172897104,-1.21283341310093,3,0,Beta-lactamase AmpC,Focal adhesion kinase 1; Protein tyrosine kina...
19038,452435,Ethyl bromopyruvate,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,4,3.60599627813507,...,23.610999999999997,12.393999999999998,-3.97725691701837,-16.185119812328,-12.4495632180388,-11.3566464088168,0,0,\N,Focal adhesion kinase 1; Protein tyrosine kina...
19039,452436,Tranilast (trans-),717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,0.0143773811775995,2.80276929553597,...,16.261000000000003,5.593500000000002,0.117052739320883,-16.5882455085828,-4.05205198147662,-1.24380427524503,0,0,\N,Focal adhesion kinase 1; Protein tyrosine kina...
19040,452437,Lenalidomide,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,7.63130256655613e-9,3.92632288853048,...,27.896500000000003,13.9555,2.60635692811078,-10.877568707302,0.456392307773086,3.93672590627491,4,0,Cereblon/Aiolos; Cereblon/Ikaros; CRL4(CRBN) E...,Focal adhesion kinase 1; Protein tyrosine kina...
19041,452438,Pomalidomide,717906-29-1,A-673,NCATS_ES(FAKI/AURKI),bone,uM,uM,2.0749974769215,3.42802026134158,...,29.292,20.989,2.67416633512853,-1.90132610098084,3.04545980488487,5.29296878563941,4,0,Cereblon/Aiolos; Cereblon/Ikaros; CRL4(CRBN) E...,Focal adhesion kinase 1; Protein tyrosine kina...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431628,1197317,AZD4547,Carfilzomib,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,3,0.0100797129163363,...,30.5715,12.1875,-3.50076669103695,-1.73253531951834,1.56737057798345,-3.5390417053228,3,4,Fibroblast growth factor receptor 3; Fibroblas...,Proteasome subunit beta type-8; 26S proteasome...
1431629,1197428,AZD4547,Cediranib,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,1.16860926806863,0.87113749279799,...,29.182,28.4455,1.97115018593238,1.86716268422302,5.25166189559464,4.0617907911506,3,3,Fibroblast growth factor receptor 3; Fibroblas...,Vascular endothelial growth factor receptor 2;...
1431630,1197492,AZD4547,AZ13401366,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,1.8028656362612,1.19527523576039,...,48.2525,44.973,19.2763035163121,11.5759177257291,16.0039359531155,21.5099028273959,3,\N,Fibroblast growth factor receptor 3; Fibroblas...,\N
1431631,1198955,AZD5582,TNF-Alpha,SW780FGF,ASTRAZENECA,urinary_tract,uM,uM,0.0635548263044466,0.000388329416594556,...,6.45399999999998,1.67249999999999,-3.15839510737771,3.92332376545969,4.5004555658562,-3.33973249214747,0,\N,Inhibitor of apoptosis protein 3; Baculoviral ...,\N


In [5]:
len(set(subdf_dc.cell_line_name))

288

In [6]:
# raw_cell_name = list(set(subdf_dc['cell_line_name']))
# for rawCellname in raw_cell_name:
#     try:
#         response = requests.get("https://web.expasy.org/cgi-bin/cellosaurus/search?input="+rawCellname)
#         getID = re.findall(r'CVCL.*">', response.text)[0]
#         getID = getID.split('"')[0]
#         print(rawCellname, getID)
#     except:
#         pass

In [19]:
###transfer cell_line_name to deepcells
sample_info = pd.read_csv('data/sample_info.csv')
drugcomb_cellosaurus = pd.read_csv("data/drugcomb_cellosaurus.csv",sep=',')

In [20]:
drugcomb_cellosaurus

,oldId,realID
0,VM-CUB-1,CVCL_1786
1,Huh-7,CVCL_0336
2,SNB-75,CVCL_1706
3,HT144,CVCL_0318
4,WM115,CVCL_0040
...,...,...
228,NCI-H3122,CVCL_5160
229,HCC1569,CVCL_1255
230,UO-31,CVCL_1911
231,MeWo,CVCL_0445


In [21]:
drugcomb_cellosaurus['realID'][125] = 'CVCL_YQ49'
drugcomb_cellosaurus['realID'][60] = 'nan'
drugcomb_cellosaurus['realID'][223] = 'CVCL_0292'
drugcomb_cellosaurus['realID'][75] = 'CVCL_0105'
drugcomb_cellosaurus['realID'][162] = 'CVCL_0021'
drugcomb_cellosaurus['realID'][70] = 'CVCL_0132'
drugcomb_cellosaurus['realID'][19] = 'CVCL_1452'
drugcomb_cellosaurus['realID'][190] = 'CVCL_0031'
drugcomb_cellosaurus['realID'][52] = 'CVCL_0291'
drugcomb_cellosaurus['realID'][73] = 'CVCL_0465'
drugcomb_cellosaurus['realID'][106] = 'CVCL_0248'

In [26]:
almanac_cellosaurus = pd.read_csv("data/almanac_cellosaurus.csv")
almanac_cellosaurus['oldId'][41] = 'MCF7'
almanac_cellosaurus['realID'][18] = 'CVCL_0292'
almanac_cellosaurus['realID'][48] = 'CVCL_0105'
almanac_cellosaurus['realID'][51] = 'CVCL_0021'
almanac_cellosaurus['realID'][17] = 'CVCL_0320'
almanac_cellosaurus['realID'][41] = 'CVCL_0031'
almanac_cellosaurus['realID'][43] = 'CVCL_0291'
almanac_cellosaurus['realID'][25] = 'CVCL_0465'
almanac_cellosaurus['realID'][0] = 'CVCL_1452'

In [27]:
drugcomb_cellosaurus_1 = pd.concat([drugcomb_cellosaurus, almanac_cellosaurus]).drop_duplicates()

In [28]:
pd.set_option('display.max_rows', 20)
drugcomb_cellosaurus_1

,oldId,realID
0,VM-CUB-1,CVCL_1786
1,Huh-7,CVCL_0336
2,SNB-75,CVCL_1706
3,HT144,CVCL_0318
4,WM115,CVCL_0040
...,...,...
229,HCC1569,CVCL_1255
230,UO-31,CVCL_1911
231,MeWo,CVCL_0445
232,U3068MG,CVCL_IR89


In [29]:
sample_info = sample_info[['DepMap_ID','RRID']]

In [30]:
realsubdf_dc = drugcomb_cellosaurus_1.merge(subdf_dc,left_on="oldId",right_on="cell_line_name")

In [31]:
drugcomb_drug = set(realsubdf_dc['realID'])
COSMIC_drug = set(sample_info['RRID'])
drugcomb_drug - (drugcomb_drug.intersection(COSMIC_drug))

{'CVCL_0125',
 'CVCL_0219',
 'CVCL_0289',
 'CVCL_0395',
 'CVCL_0417',
 'CVCL_0B75',
 'CVCL_1384',
 'CVCL_1452',
 'CVCL_1570',
 'CVCL_1813',
 'CVCL_2005',
 'CVCL_2088',
 'CVCL_2220',
 'CVCL_2H54',
 'CVCL_3799',
 'CVCL_4633',
 'CVCL_4791',
 'CVCL_5246',
 'CVCL_5310',
 'CVCL_5997',
 'CVCL_9827',
 'CVCL_A426',
 'CVCL_A442',
 'CVCL_A794',
 'CVCL_A8EZ',
 'CVCL_A8IW',
 'CVCL_AP82',
 'CVCL_AX39',
 'CVCL_IP22',
 'CVCL_IR58',
 'CVCL_IR61',
 'CVCL_IR67',
 'CVCL_IR73',
 'CVCL_IR89',
 'CVCL_IT41',
 'CVCL_JF18',
 'CVCL_JF68',
 'CVCL_M814',
 'CVCL_T887',
 'CVCL_VT28',
 'CVCL_VT33',
 'CVCL_Y053',
 'CVCL_YQ49',
 'CVCL_YX83',
 'CVCL_YX86',
 'CVCL_ZA06',
 'nan'}

In [34]:
len(drugcomb_drug.intersection(COSMIC_drug))

185

In [35]:
realsubdf_dc = realsubdf_dc.merge(sample_info, left_on='realID',right_on='RRID')

In [36]:
realsubdf_dc

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,synergy_hsa,synergy_bliss,drug_row_clinical_phase,drug_col_clinical_phase,drug_row_target_name,drug_col_target_name,DepMap_ID_x,RRID_x,DepMap_ID_y,RRID_y
0,VM-CUB-1,CVCL_1786,1187287,AZD5363,KU-60019,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,6.32323598003741,5.24513954350488,2,0,Rho-associated protein kinase 1; Rho-associate...,Serine-protein kinase ATM,ACH-000545,CVCL_1786,ACH-000545,CVCL_1786
1,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-22.8113258572499,-21.7616990206227,2,2,Rho-associated protein kinase 1; Rho-associate...,Serine-protein kinase ATR; Serine/threonine-pr...,ACH-000545,CVCL_1786,ACH-000545,CVCL_1786
2,VM-CUB-1,CVCL_1786,1187528,AZD5363,AZD-4877,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-40.2879401730772,29.9623708194271,2,2,Rho-associated protein kinase 1; Rho-associate...,Kinesin-like protein 1,ACH-000545,CVCL_1786,ACH-000545,CVCL_1786
3,VM-CUB-1,CVCL_1786,1187552,AZD5363,Sapitinib,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,1.47603779864746,-8.33016270978925,2,2,Rho-associated protein kinase 1; Rho-associate...,Receptor tyrosine-protein kinase erbB-3; Recep...,ACH-000545,CVCL_1786,ACH-000545,CVCL_1786
4,VM-CUB-1,CVCL_1786,1187624,AZD5363,AZD4547,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,-0.776249738659982,146.089535158276,2,3,Rho-associated protein kinase 1; Rho-associate...,Fibroblast growth factor receptor 3; Fibroblas...,ACH-000545,CVCL_1786,ACH-000545,CVCL_1786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530496,MeWo,CVCL_0445,1414780,ZM 336372,CHEMBL1254007,MeWo,FRIEDMAN,skin,uM,uM,...,-47.4546017047357,-53.2051217717236,0,0,MAP kinase p38 alpha; Serine/threonine-protein...,Vascular endothelial growth factor receptor 2;...,ACH-000987,CVCL_0445,ACH-000987,CVCL_0445
530497,MeWo,CVCL_0445,1414816,ZM 336372,Tipifarnib,MeWo,FRIEDMAN,skin,uM,uM,...,-18.9127303185778,-13.3089508855435,0,3,MAP kinase p38 alpha; Serine/threonine-protein...,Protein farnesyltransferase; Protein farnesylt...,ACH-000987,CVCL_0445,ACH-000987,CVCL_0445
530498,MeWo,CVCL_0445,1414852,ZM 336372,112648-68-7,MeWo,FRIEDMAN,skin,uM,uM,...,-14.2853522779673,-13.9084149740677,0,0,MAP kinase p38 alpha; Serine/threonine-protein...,Aldehyde dehydrogenase 1A1; Telomerase reverse...,ACH-000987,CVCL_0445,ACH-000987,CVCL_0445
530499,MeWo,CVCL_0445,1414888,ZM 336372,vasastrol,MeWo,FRIEDMAN,skin,uM,uM,...,-22.5798008488644,-29.4067097567556,0,\N,MAP kinase p38 alpha; Serine/threonine-protein...,\N,ACH-000987,CVCL_0445,ACH-000987,CVCL_0445


#### GraphSynergy: drugs not in drug_protein.csv

In [37]:
###1. map pubchemID to comdb

def get_uniquedrug(subsetdf):
    drug_list = []
    drug1_dc = subsetdf["drug_row"].unique()
    drug2_dc = subsetdf["drug_col"].unique()
    
    drug_list.append(set(drug1_dc))
    drug_list.append(set(drug2_dc))
    return drug_list

In [44]:
#drug_list = get_uniquedrug(subdf_dc)
drug_list = get_uniquedrug(realsubdf_dc)
# One-Liner to union a list of sets
drug_list_set = set().union(*drug_list)

In [45]:
len(drug_list_set)

3127

In [17]:
def get_compounds(a_drug):
    return pcp.get_compounds(a_drug, 'name',listkey_count=1)

def get_pubchemID(drug_list_set):
    ##too slow
    #df["drug1_pubchemID"] = df[drug1].apply(lambda x: get_compounds(x))
    #df["drug2_pubchemID"] = df[drug2].apply(lambda x: get_compounds(x))
    chemID_list = [ get_compounds(x)[0] if len(get_compounds(x))>0 else "" for x in drug_list_set ]
    
    print(x for x in drug_list_set)
    return chemID_list

In [18]:
# chemID_list = get_pubchemID(drug_list_set)
# pubchemIDmapping = pd.DataFrame({
#     'drug_name': list(drug_list_set),
#     'pubchemID': chemID_list
# })
# pubchemIDmapping['pubchemID'] = pubchemIDmapping['pubchemID'].astype(str)

In [46]:
# pubchemIDmapping.to_csv('chemID_drugcomb.csv')
pubchemIDmapping = pd.read_csv('chemID_drugcomb.csv')
pubchemIDmapping = pubchemIDmapping[['drug_name','pubchemID']]

In [47]:
pubchemIDmapping['pubchemID'][12] = 'Compound(57519511)'
pubchemIDmapping['pubchemID'][446] = 'Compound(53396327)'
pubchemIDmapping['pubchemID'][554] = 'Compound(60865)'
pubchemIDmapping['pubchemID'][560] = 'Compound(15604015)'
pubchemIDmapping['pubchemID'][715] = 'Compound(5284419)'
pubchemIDmapping['pubchemID'][764] = 'Compound(363740391)'
pubchemIDmapping['pubchemID'][965] = 'Compound(163176602)'
pubchemIDmapping['pubchemID'][1565] = 'Compound(73265275)'
pubchemIDmapping['pubchemID'][1690] = 'Compound(73265237)'
pubchemIDmapping['pubchemID'][1861] = 'Compound(26041)'
pubchemIDmapping['pubchemID'][1883] = 'Compound(9578866)'
pubchemIDmapping['pubchemID'][2397] = 'Compound(6445562)'
pubchemIDmapping['pubchemID'][3038] = 'Compound(6918664)'

In [48]:
pubchemIDmapping = pubchemIDmapping[pubchemIDmapping['pubchemID'].notna()]

In [49]:
pubchemIDmapping

,drug_name,pubchemID
0,primaquine diphosphate,Compound(6135)
1,Gbr 12935,Compound(3456)
2,GW7647,Compound(3392731)
3,153436-54-5,Compound(4705)
4,Rizatriptan benzoate,Compound(77997)
...,...,...
3122,KETOTIFEN FUMARATE,Compound(5282408)
3123,Ki8751,Compound(11317348)
3124,Fesoterodine fumarate,Compound(9849808)
3125,Rifaximin,Compound(6436173)


In [65]:
#re.search(r'\((.*?)\)','Compound(148124)').group(1)''
def split_it(compound):
    return int(re.search(r'\((.*?)\)', compound).group(1))

In [66]:
pubchemIDmapping['cleanID'] = pubchemIDmapping['pubchemID'].apply(lambda x: split_it(x))

In [67]:
pubchemIDmapping['cleanID'] = pubchemIDmapping['cleanID'].astype(int)

In [42]:
def get_pubchemsdf(cleanID):
    pcp.download('SDF', str(cleanID)+'.sdf', cleanID, 'cid', overwrite=True)

In [45]:
# for _ in list(pubchemIDmapping['cleanID']):
#     try:
#         get_pubchemsdf(_)
#     except:
#         print(_)
#         pass

In [23]:
# Calculate drug similarity against all drugbank drugs

In [50]:
# Read SDF File
supplier = rdkit.Chem.SDMolSupplier('../ChemSim/structures.sdf')
molecules = [mol for mol in supplier if mol is not None]
drugcomb_supplier = rdkit.Chem.SDMolSupplier('sdf_drugcomb/drugcomb_structures.sdf')
drugcomb_molecules = [mol for mol in drugcomb_supplier if mol is not None]

RDKit ERROR: [22:51:47] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[22:51:47] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [22:51:47] ERROR: Could not sanitize molecule ending on line 289199
[22:51:47] ERROR: Could not sanitize molecule ending on line 289199
RDKit ERROR: [22:51:47] ERROR: Explicit valence for atom # 13 Cl, 5, is greater than permitted
[22:51:47] ERROR: Explicit valence for atom # 13 Cl, 5, is greater than permitted
[22:51:47] Explicit valence for atom # 19 O, 3, is greater than permitted
RDKit ERROR: [22:51:47] Explicit valence for atom # 19 O, 3, is greater than permitted
RDKit ERROR: [22:51:47] ERROR: Could not sanitize molecule ending on line 399275
[22:51:47] ERROR: Could not sanitize molecule ending on line 399275
RDKit ERROR: [22:51:47] ERROR: Explicit valence for atom # 19 O, 3, is greater than permitted
[22:51:47] ERROR: Explicit valence for atom # 19 O, 3, is greater than permitted
[22:51:47] Explicit valenc

In [51]:
len(drugcomb_molecules)

3043

In [48]:
# Calculate fingerprints for drugbank
fingerprints = dict()
for mol in molecules:
    drugbank_id = mol.GetProp('DATABASE_ID')
    fingerprint = rdkit.Chem.AllChem.GetMorganFingerprint(mol, 2)
    fingerprints[drugbank_id] = fingerprint

In [49]:
drugcomb_fingerprints = dict()
for mol in drugcomb_molecules:
    pubchem_id = mol.GetProp('PUBCHEM_COMPOUND_CID')
    fingerprint = rdkit.Chem.AllChem.GetMorganFingerprint(mol, 2)
    drugcomb_fingerprints[pubchem_id] = fingerprint

In [50]:
# Calculate pairwise compound similarities
similarity_rows = list()
for id0, fp0 in fingerprints.items():
    for id1,fp1 in drugcomb_fingerprints.items():
        similarity = rdkit.DataStructs.DiceSimilarity(fp0, fp1)
        similarity = round(similarity, 4)
        similarity_rows.append([id0, id1, similarity])

In [51]:
similarity_df = pd.DataFrame(similarity_rows, columns=['compound0', 'compound1', 'similarity'])

In [58]:
#similarity_df.to_csv("similarity_drugcomb.tsv", sep='\t')
similarity_df = pd.read_csv("similarity_drugcomb.tsv", sep='\t')

In [59]:
len(similarity_df)

34355470

In [60]:
pd.set_option('display.max_rows', 200)
idx = similarity_df.groupby(by=["compound1"])['similarity'].transform(max) == similarity_df['similarity']
similarity_df = similarity_df[idx]
similarity_df = similarity_df.drop_duplicates(subset=['compound1'])

In [61]:
confident_similarity_df = similarity_df[similarity_df['similarity']>=0.8]
confident_similarity_df

,Unnamed: 0,compound0,compound1,similarity
21871,21871,DB00080,16134395,1.0000
26190,26190,DB00091,5284373,1.0000
31947,31947,DB00104,448601,1.0000
41453,41453,DB00115,5311498,0.9355
42307,42307,DB00115,73265345,0.9355
...,...,...,...,...
34055141,34055141,DB16305,25022340,1.0000
34127577,34127577,DB16346,123723,1.0000
34221434,34221434,DB16567,9576185,0.9925
34320126,34320126,DB16702,3326,1.0000


In [62]:
notconfident_similarity_df = similarity_df[similarity_df['similarity']<0.8]
notconfident_similarity_df.head(20)

,Unnamed: 0,compound0,compound1,similarity
230846,230846,DB00179,72300,0.5517
240514,240514,DB00183,108187,0.5252
243510,243510,DB00184,10434468,0.5932
297473,297473,DB00201,6241,0.6981
323127,323127,DB00210,16124726,0.6264
343506,343506,DB00216,73265274,0.6012
393135,393135,DB00233,16367,0.6744
400192,400192,DB00235,4593,0.7475
458470,458470,DB00254,54686183,0.5670
473577,473577,DB00259,5324,0.7576


In [68]:
pubchemIDmapping[pubchemIDmapping['cleanID'] == 16760703]

,drug_name,pubchemID,cleanID
950,Sodium valproate,Compound(16760703),16760703


In [69]:
pubchemIDmapping['cleanID'] = pubchemIDmapping['cleanID'].astype(int)
confident_similarity_df['compound1'] = confident_similarity_df['compound1'].astype(int)

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [70]:
confident_similarity =  pd.merge(pubchemIDmapping, confident_similarity_df, left_on=['cleanID'],right_on=['compound1'])

In [87]:
confident_similarity[confident_similarity.duplicated(subset=['drug_name'])]
confident_similarity

,drug_name,pubchemID,cleanID,Unnamed: 0,compound0,compound1,similarity
0,primaquine diphosphate,Compound(6135),6135,2923567,DB01087,6135,0.8438
1,153436-54-5,Compound(4705),4705,9243218,DB03365,4705,0.9048
2,Rizatriptan benzoate,Compound(77997),77997,2534600,DB00953,77997,0.8227
3,triamterene,Compound(5546),5546,844971,DB00384,5546,1.0000
4,DOXOFYLLINE,Compound(50942),50942,22723809,DB09273,50942,1.0000
...,...,...,...,...,...,...,...
2158,Tariquidar,Compound(148201),148201,14789461,DB06240,148201,1.0000
2159,KETOTIFEN FUMARATE,Compound(5282408),5282408,2436232,DB00920,5282408,0.8649
2160,Fesoterodine fumarate,Compound(9849808),9849808,15461397,DB06702,9849808,0.8913
2161,Rifaximin,Compound(6436173),6436173,3325304,DB01220,6436173,1.0000


In [72]:
temp1_dc = pd.merge(realsubdf_dc, confident_similarity, left_on=['drug_row'],right_on=['drug_name'], how='inner')
#temp1_dc = temp1_dc[list(realsubdf_dc.columns)]
temp2_dc = pd.merge(temp1_dc, confident_similarity, left_on=['drug_col'],right_on=['drug_name'], how='inner')

#drug_comb = temp2_dc[list(realsubdf_dc.columns)]
#drug_comb.columns = ['Drug1','Drug2','cell','drug1_db','drug2_db','synergy']

In [92]:
temp2_dc.to_csv('drugcomb_trueset.csv')

In [89]:
temp2_dc

,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,conc_col_unit,...,compound0_x,compound1_x,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y
0,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
1,SW780,CVCL_1728,1187297,AZD5363,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
2,KU-19-19,CVCL_1344,1187294,AZD5363,AZD6738,KU-19-19,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
3,T24,CVCL_0554,1187298,AZD5363,AZD6738,T24,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
4,BFTC-905,CVCL_1083,1187290,AZD5363,AZD6738,BFTC-905,ASTRAZENECA,urinary_tract,uM,uM,...,DB12218,25227436,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373819,COLO 205,CVCL_0218,1179718,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
373820,COLO 205,CVCL_0218,1179889,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
373821,COLO 205,CVCL_0218,1180060,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000
373822,MDA-MB-468,CVCL_0419,1181577,5Z-7-Oxozeaenol,Doramapimod,MDA-MB-468,FLOBAK,breast,ÂµM,ÂµM,...,DB07788,9863776,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000


In [90]:
len(set(temp2_dc['realID']))

184

In [91]:
temp_list = []
temp_list.append(temp2_dc['compound0_x'].to_list())
temp_list.append(temp2_dc['compound0_y'].to_list())
len(set().union(*temp_list))

1349

In [13]:
#TGSA_drugcomb = pd.read_csv('/home/cx79/scratch60/GraphSynergy_pivotals/data/DrugComb-trueset/drug_combinations_whole_TGSA.csv')

#### Remove duplicates

In [4]:
dc_df =  pd.read_csv('drugcomb_trueset.csv')

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (22,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
len(dc_df)

373824

In [6]:
dc_df['synergistic'] = [0] * len(dc_df)
dc_df = dc_df[dc_df['synergy_loewe'] != '\\N']
dc_df['synergy_loewe'] = dc_df['synergy_loewe'].astype(float)
dc_df.loc[dc_df['synergy_loewe'] > 0, 'synergistic'] = 1

In [7]:
def combine_triplets(test_set):
    combined_list =[]
    for index, row in test_set.iterrows():
        combined_list.append((row['realID'],row['compound0_x'],row['compound0_y']))

    test_set['combined'] = combined_list
    return test_set

In [8]:
combined_dc_df = combine_triplets(dc_df)

In [9]:
#combined_dc_df[combined_dc_df.duplicated(subset='combined')].to_csv('duplciates_in_dc.csv')
nondup = combined_dc_df[~combined_dc_df.duplicated(subset='combined',keep=False)]
nondup

,Unnamed: 0,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
0,0,VM-CUB-1,CVCL_1786,1187301,AZD5363,AZD6738,VM-CUB-1,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1786, DB12218, DB14917)"
1,1,SW780,CVCL_1728,1187297,AZD5363,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1728, DB12218, DB14917)"
2,2,KU-19-19,CVCL_1344,1187294,AZD5363,AZD6738,KU-19-19,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1344, DB12218, DB14917)"
3,3,T24,CVCL_0554,1187298,AZD5363,AZD6738,T24,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_0554, DB12218, DB14917)"
4,4,BFTC-905,CVCL_1083,1187290,AZD5363,AZD6738,BFTC-905,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1083, DB12218, DB14917)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373763,373763,SMS-CTR,CVCL_A770,462367,CHEMBL3348822,ceritinib,SMS-CTR,YOHE,soft_tissue,uM,...,1.0,ceritinib,Compound(57379345),57379345,22182559,DB09063,57379345,1.0000,1,"(CVCL_A770, DB01590, DB09063)"
373764,373764,RD,CVCL_1649,462553,Omipalisib,Omipalisib,RD,YOHE,soft_tissue,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB12703, DB12703)"
373765,373765,SMS-CTR,CVCL_A770,462373,Omipalisib,Omipalisib,SMS-CTR,YOHE,soft_tissue,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,1,"(CVCL_A770, DB12703, DB12703)"
373766,373766,RD,CVCL_1649,462556,CHEMBL3348822,Omipalisib,RD,YOHE,soft_tissue,uM,...,1.0,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB01590, DB12703)"


In [28]:
dupl = combined_dc_df[combined_dc_df.duplicated(subset='combined',keep=False)]
dupl

,Unnamed: 0,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
95,95,SW780,CVCL_1728,1195616,AZD4547,AZD6738,SW780,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,1,"(CVCL_1728, DB12247, DB14917)"
102,102,RT-112,CVCL_1670,1195615,AZD4547,AZD6738,RT-112,ASTRAZENECA,urinary_tract,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1670, DB12247, DB14917)"
258,258,HCC1187,CVCL_1247,1192592,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
259,259,HCC1187,CVCL_1247,1192593,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
260,260,HCC1187,CVCL_1247,1192594,Gefitinib,AZD6738,HCC1187,ASTRAZENECA,breast,uM,...,1.0,AZD6738,Compound(54761306),54761306,31585328,DB14917,54761306,0.8659,0,"(CVCL_1247, DB00317, DB14917)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373819,373819,COLO 205,CVCL_0218,1179718,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
373820,373820,COLO 205,CVCL_0218,1179889,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
373821,373821,COLO 205,CVCL_0218,1180060,5Z-7-Oxozeaenol,Doramapimod,COLO 205,FLOBAK,large_intestine,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0218, DB07788, DB03044)"
373822,373822,MDA-MB-468,CVCL_0419,1181577,5Z-7-Oxozeaenol,Doramapimod,MDA-MB-468,FLOBAK,breast,ÂµM,...,1.0,Doramapimod,Compound(156422),156422,8317047,DB03044,156422,1.0000,0,"(CVCL_0419, DB07788, DB03044)"


In [45]:
# groupdicts = dupl.groupby(by=["combined"]).groups
# groupdicts

In [46]:
def summarize_dupl(dupl):
    lists_ix = []
    groupdicts = dupl.groupby(by=["combined"]).groups
    for key, ind_list in groupdicts.items():
        subdf = dupl.loc[ind_list]
        if len(subdf['study_name']) <= 1:
            ix = subdf.drop_duplicates(subset=['combined']).index[0]
        
        elif subdf['synergistic'].mean() == 0 or subdf['synergistic'].mean() == 1:
            ## keep either row
            ix = subdf.drop_duplicates(subset=['combined']).index[0]
        else:
            continue
        lists_ix.append(ix)
    
    return dupl.loc[lists_ix]

In [47]:
dupl_slim = summarize_dupl(dupl)

In [51]:
combined_dc_df_NoDup = pd.concat([dupl_slim, nondup])

In [52]:
combined_dc_df_NoDup.to_csv('drugcomb_trueset_NoDup.csv')

In [53]:
combined_dc_df_NoDup

,Unnamed: 0,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
226051,226051,K-562,CVCL_0004,289684,CHEMBL17639,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00242, DB00445)"
227093,227093,K-562,CVCL_0004,161172,5-Fluoro-2'-deoxyuridine,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00322, DB00445)"
224413,224413,K-562,CVCL_0004,199141,Zoledronic acid,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00399, DB00445)"
228732,228732,K-562,CVCL_0004,223458,122111-05-1,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,0.9897,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00441, DB00445)"
234396,234396,K-562,CVCL_0004,434620,ADM hydrochloride,NSC256439,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,0.9953,NSC256439,Compound(3685),3685,3193352,DB01177,3685,1.0000,0,"(CVCL_0004, DB00445, DB01177)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373763,373763,SMS-CTR,CVCL_A770,462367,CHEMBL3348822,ceritinib,SMS-CTR,YOHE,soft_tissue,uM,...,1.0000,ceritinib,Compound(57379345),57379345,22182559,DB09063,57379345,1.0000,1,"(CVCL_A770, DB01590, DB09063)"
373764,373764,RD,CVCL_1649,462553,Omipalisib,Omipalisib,RD,YOHE,soft_tissue,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB12703, DB12703)"
373765,373765,SMS-CTR,CVCL_A770,462373,Omipalisib,Omipalisib,SMS-CTR,YOHE,soft_tissue,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,1,"(CVCL_A770, DB12703, DB12703)"
373766,373766,RD,CVCL_1649,462556,CHEMBL3348822,Omipalisib,RD,YOHE,soft_tissue,uM,...,1.0000,Omipalisib,Compound(25167777),25167777,26891947,DB12703,25167777,1.0000,0,"(CVCL_1649, DB01590, DB12703)"


In [54]:
dupl_slim

,Unnamed: 0,oldId,realID,block_id,drug_row,drug_col,cell_line_name,study_name,tissue_name,conc_row_unit,...,similarity_x,drug_name_y,pubchemID_y,cleanID_y,Unnamed: 0_y,compound0_y,compound1_y,similarity_y,synergistic,combined
226051,226051,K-562,CVCL_0004,289684,CHEMBL17639,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00242, DB00445)"
227093,227093,K-562,CVCL_0004,161172,5-Fluoro-2'-deoxyuridine,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00322, DB00445)"
224413,224413,K-562,CVCL_0004,199141,Zoledronic acid,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,1.0000,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00399, DB00445)"
228732,228732,K-562,CVCL_0004,223458,122111-05-1,ADM hydrochloride,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,0.9897,ADM hydrochloride,Compound(32874),32874,1023615,DB00445,32874,0.9953,0,"(CVCL_0004, DB00441, DB00445)"
234396,234396,K-562,CVCL_0004,434620,ADM hydrochloride,NSC256439,K-562,ALMANAC,haematopoietic_and_lymphoid,uM,...,0.9953,NSC256439,Compound(3685),3685,3193352,DB01177,3685,1.0000,0,"(CVCL_0004, DB00445, DB01177)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373706,373706,EW-8,CVCL_V618,461857,ZINC34894448,PF-3758309,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,...,1.0000,PF-3758309,Compound(25227462),25227462,24542764,DB11775,25227462,1.0000,1,"(CVCL_V618, DB12021, DB11775)"
373562,373562,EW-8,CVCL_V618,461846,MK-5108,Doxorubicin Hydrochloride,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,...,1.0000,Doxorubicin Hydrochloride,Compound(443939),443939,1023908,DB00445,443939,0.9953,1,"(CVCL_V618, DB12556, DB00445)"
7264,7264,EW-8,CVCL_V618,461874,10356-76-0,Olaparib,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,...,1.0000,Olaparib,Compound(23725625),23725625,22211694,DB09074,23725625,1.0000,0,"(CVCL_V618, DB12957, DB09074)"
259492,259492,EW-8,CVCL_V618,461873,10356-76-0,Niraparib,EW-8,NCATS_ES(NAMPT+PARP),bone,uM,...,1.0000,Niraparib,Compound(24958200),24958200,24588355,DB11793,24958200,1.0000,0,"(CVCL_V618, DB12957, DB11793)"


In [181]:
# #To summarize both anchor concentrations, 
# #we considered a ‘combination–cell line’ pair as synergistic if synergy was observed at either anchor concentration.
# def summarize_anchor(temp2_dc):
#     idx = temp2_dc.groupby(by=["combined"])['synergistic'].transform(any)
#     temp2_dc_synergyTrue = temp2_dc[idx]
#     temp2_dc_synergyFalse = temp2_dc[~idx]
#     temp2_dc_synergyTrue = temp2_dc_synergyTrue.drop_duplicates(subset=['combined'])
#     temp2_dc_synergyFalse = temp2_dc_synergyFalse.drop_duplicates(subset=['combined'])
    
#     temp2_dc_synergyTrue['synergistic'] =  temp2_dc_synergyTrue['synergistic'].apply(lambda x: True)
#     temp2_dc_synergyFalse['synergistic'] =  temp2_dc_synergyFalse['synergistic'].apply(lambda x: False)
#     temp3 = pd.concat([temp2_dc_synergyTrue, temp2_dc_synergyFalse])
#     return temp3

In [182]:
# combined_dc_df_NoDup = summarize_anchor(combined_dc_df)

#### process CCLE, select 706 cancer-related genes, and 57 cell lines

In [39]:
mu_original = pd.read_csv("data/CellLines_DepMap/CCLE_580_18281/census_706/mu.csv")
genes_706 = list(mu_original.columns)[1:]
clean_genes_706 = [int(split_it(_)) for _ in genes_706]
clean_cells_57 = list(set(list(temp2_dc['DepMap_ID'])))

In [3]:
### look at if alamanc has cell line that all in TGSA
CCLE_mu = pd.read_csv('data/CCLE_mutations.csv')
CCLE_ex = pd.read_csv('data/CCLE_expression.csv')
CCLE_cn = pd.read_csv('data/CCLE_gene_cn.csv')

/gpfs/ycga/project/christakis/cx79/conda_envs/meta/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3186: DtypeWarning: Columns (3,19,22,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
CCLE_mu

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,Variant_annotation,CGA_WES_AC,HC_AC,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,False,0.0,NaN,damaging,34:213,NaN,NaN,NaN,34:221,NaN
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,False,3.0,NaN,other non-conserving,57:141,NaN,NaN,NaN,9:0,28:32
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,False,0.0,NaN,silent,118:0,NaN,NaN,10:0,118:0,18:0
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,False,0.0,NaN,damaging,NaN,NaN,NaN,6:28,NaN,NaN
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,False,0.0,NaN,other non-conserving,28:62,NaN,NaN,NaN,27:61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230240,SLC39A4,55630,37,8,145641991,145641991,+,Silent,SNP,C,...,False,0.0,0.000017,silent,52:23,NaN,NaN,NaN,NaN,NaN
1230241,TAL2,6887,37,9,108424778,108424778,+,Start_Codon_SNP,SNP,A,...,False,0.0,NaN,damaging,27:0,NaN,NaN,NaN,NaN,NaN
1230242,TRO,7216,37,X,54955098,54955098,+,Silent,SNP,C,...,False,0.0,NaN,silent,5:16,NaN,NaN,NaN,NaN,NaN
1230243,USP51,158880,37,X,55514703,55514703,+,Missense_Mutation,SNP,G,...,False,0.0,NaN,other non-conserving,23:0,NaN,NaN,NaN,NaN,NaN


In [41]:
def one_hot_706_mu(CCLE_mu):
    temp1 = CCLE_mu[CCLE_mu.Entrez_Gene_Id.isin(clean_genes_706)]
    temp2 = temp1[temp1.DepMap_ID.isin(clean_cells_57)]
    onehot_rows = list()
    for cell in clean_cells_57:
        onehot_row = list()
        temp_cell = temp2[temp2['DepMap_ID']==cell]
        temp_gene_list = list(temp_cell['Entrez_Gene_Id'])
        onehot_row.append(cell)
        for gene in clean_genes_706:
            if gene in temp_gene_list:
                onehot_row.append(1)
            else:
                onehot_row.append(0)
        onehot_rows.append(onehot_row)
    return onehot_rows

In [42]:
 pd.DataFrame(one_hot_706_mu(CCLE_mu), columns=['DepMap_ID']+clean_genes_706).to_csv('data/almanac_mu.csv')

In [43]:
 pd.DataFrame(one_hot_706_mu(CCLE_mu), columns=['DepMap_ID']+clean_genes_706)

,DepMap_ID,10000,10006,10019,100533467,1008,1009,10142,1015,10186,...,974,9774,9817,9869,9901,9935,9950,9967,9968,999
0,ACH-000343,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ACH-002199,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,ACH-000971,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
3,ACH-001151,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ACH-000966,0,0,0,0,1,1,1,0,0,...,0,1,0,1,1,0,0,0,1,1
5,ACH-002172,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ACH-000288,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ACH-000579,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,ACH-000232,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ACH-000090,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
## not use exp or cnv

In [45]:
CCLE_ex.columns = [None]+[split_it(_) for _ in list(CCLE_ex.columns)[1:]]
def get_706_ex(CCLE_ex):
    temp1 = CCLE_ex[[None]+clean_genes_706]
    temp2 = temp1[temp1.iloc[:,0].isin(clean_cells_57)]
    return temp2

In [46]:
get_706_ex(CCLE_ex)

,NaN,10000,10006,10019,100533467,1008,1009,10142,1015,10186,...,974,9774,9817,9869,9901,9935,9950,9967,9968,999
83,ACH-000376,2.731183,3.958843,4.819157,0.422233,0.584963,0.464668,4.997292,0.097611,4.393004,...,0.042644,5.926948,5.896272,4.381283,1.395063,0.163499,4.050502,5.976364,3.604071,0.400538
93,ACH-000552,0.084064,4.477030,1.378512,0.555816,0.028569,0.356144,4.858976,1.070389,0.084064,...,0.070389,5.625563,5.170326,4.890447,1.063503,0.565597,3.771886,6.027021,4.195348,6.795585
139,ACH-000971,0.389567,4.465974,3.524816,0.389567,0.042644,0.070389,4.082362,0.150560,0.378512,...,0.275007,6.864681,6.100347,5.039577,1.682573,0.137504,4.465974,6.922317,3.807355,4.914086
144,ACH-000019,0.070389,4.438958,0.879706,0.505891,0.014355,0.516015,4.824768,0.042644,0.056584,...,0.000000,5.875043,5.584061,5.974300,2.295723,2.179511,5.236875,5.714520,3.791814,8.279750
161,ACH-000428,3.953265,4.898208,4.657068,0.871844,1.405992,0.189034,3.764474,0.650765,4.030336,...,0.790772,5.368419,5.269781,3.915521,1.432959,0.000000,4.976822,5.612942,3.132577,0.782409
292,ACH-000681,4.148934,5.090430,1.304511,0.000000,0.275007,0.422233,4.013462,3.089159,1.275007,...,0.084064,5.828073,5.365623,4.266037,0.895303,0.014355,4.850499,5.559492,2.937344,2.893362
300,ACH-000997,0.687061,4.635754,3.099295,0.000000,0.014355,0.214125,4.856986,3.032101,0.815575,...,0.137504,6.612058,5.919578,5.081510,1.176323,0.028569,5.388534,6.866537,3.933573,7.857857
381,ACH-000148,5.499208,4.139961,3.632268,0.807355,0.084064,7.039687,4.190615,0.056584,5.495695,...,0.084064,5.293885,4.853497,3.943921,1.823749,3.135863,5.006298,5.504620,3.489286,1.021480
426,ACH-000273,4.541019,4.397803,4.126808,0.000000,0.000000,5.985273,2.803227,0.124328,5.377471,...,0.097611,5.712321,5.697663,4.819668,0.575312,3.436961,4.627023,5.911692,3.841973,0.344828
473,ACH-000551,0.516015,4.608218,4.995485,0.176323,0.028569,0.000000,4.937815,0.042644,0.014355,...,0.411426,5.542568,5.174327,5.045705,0.895303,0.000000,4.354734,5.876517,4.667892,0.910733


In [47]:
CCLE_cn.columns = [None]+[split_it(_) for _ in list(CCLE_cn.columns)[1:]]
get_706_ex(CCLE_cn)

,NaN,10000,10006,10019,100533467,1008,1009,10142,1015,10186,...,974,9774,9817,9869,9901,9935,9950,9967,9968,999
15,ACH-000477,1.169867,0.847422,1.049987,0.842412,1.011337,0.860800,1.602487,0.963662,0.842412,...,1.127401,0.859953,0.948192,1.169867,1.101864,1.224745,1.051725,0.849991,0.637492,0.860800
20,ACH-001738,0.989219,1.002143,1.001853,0.988445,1.002190,1.020875,0.995771,1.027260,0.982791,...,1.027981,0.953204,1.032320,1.014101,0.953737,1.326701,0.981153,1.004010,0.946920,1.020875
60,ACH-000463,1.382695,0.854136,1.122450,0.873213,1.175921,1.387021,1.143142,0.870934,0.873213,...,1.152464,0.861529,0.812400,1.381898,0.865442,1.139810,1.140845,0.848222,0.842764,1.387021
79,ACH-000376,1.004714,0.757291,1.001570,1.134294,1.206008,1.234097,1.412827,1.040069,1.104462,...,1.352032,0.988312,1.134948,0.969222,1.009104,1.021755,0.746772,0.969044,1.096740,1.234097
96,ACH-000615,1.007507,0.806099,0.982681,0.773477,0.916177,0.796038,1.207329,1.031615,0.796595,...,0.991071,1.009940,1.090264,0.976266,0.983808,1.183075,0.796720,1.162941,0.586753,0.796038
129,ACH-000433,1.495285,1.194374,1.079431,0.984292,1.119083,1.248368,1.114578,1.294507,0.954733,...,1.517890,0.841078,0.867295,1.381811,0.785959,1.132726,0.728257,0.964142,0.776521,1.248368
163,ACH-000019,1.033839,1.061974,1.055406,0.631680,1.301971,0.660817,1.244271,1.753673,1.052650,...,1.925169,1.054023,1.014479,1.623545,1.247515,1.075394,1.263234,0.831631,0.669850,1.044579
223,ACH-000148,0.862566,0.863635,0.876592,0.842241,1.333728,0.844306,1.128962,1.557337,0.842241,...,0.877041,0.847808,1.097797,0.863664,1.138744,1.139145,0.867974,1.137546,0.503616,0.844306
256,ACH-000825,1.245228,1.305949,0.957192,0.948613,1.286408,0.994351,1.511112,1.238408,0.948613,...,1.334678,0.878364,0.996680,1.134266,1.009662,1.227183,0.599644,1.012479,0.708793,0.994351
287,ACH-000555,1.209465,1.117880,0.922738,0.901190,1.225209,1.021839,1.480279,1.433643,0.893800,...,1.136822,0.825510,1.126812,1.207892,0.674110,1.123056,0.657615,0.911256,1.013089,1.021839


In [48]:
#### alamanc drugs to smiles

In [53]:
def get_smiles(a_drug):
    c = pcp.Compound.from_cid(a_drug)
    return c.canonical_smiles

In [54]:
smiles_list = [ get_smiles(x) for x in list(pubchemIDmapping['cleanID']) ]

In [56]:
pubchemIDmapping['smiles'] = smiles_list

In [59]:
pubchemIDmapping.to_csv('almanac_smiles.csv')

In [ ]:
### synergistic

In [3]:
IC = pd.read_csv('ALMANAC.csv')
IC['synergistic'] = [0] * len(IC)
IC.loc[IC['synergy_loewe'] > 0, 'synergistic'] = 1

In [13]:
IC.columns

Index(['Unnamed: 0', 'oldId', 'realID', 'block_id', 'drug_row', 'drug_col',
       'cell_line_name', 'study_name', 'tissue_name', 'conc_row_unit',
       'conc_col_unit', 'ic50_row', 'ic50_col', 'ri_row', 'ri_col', 'css_row',
       'css_col', 'css_ri', 'S_sum', 'S_mean', 'S_max', 'synergy_zip',
       'synergy_loewe', 'synergy_hsa', 'synergy_bliss',
       'drug_row_clinical_phase', 'drug_col_clinical_phase',
       'drug_row_target_name', 'drug_col_target_name', 'DepMap_ID', 'RRID',
       'drug_name_x', 'pubchemID_x', 'cleanID_x', 'Unnamed: 0_x',
       'compound0_x', 'compound1_x', 'similarity_x', 'drug_name_y',
       'pubchemID_y', 'cleanID_y', 'Unnamed: 0_y', 'compound0_y',
       'compound1_y', 'similarity_y', 'synergistic'],
      dtype='object')

In [6]:
set(IC['tissue_name'])

{'brain',
 'breast',
 'haematopoietic_and_lymphoid',
 'kidney',
 'large_intestine',
 'lung',
 'ovary',
 'prostate',
 'skin'}